IAMSI -- 2017-2018
--------
*&copy;Equipe pédagogique IAMSI'2018*


# TME 09 : Règles d'association

<font size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

* Adrien POUYET & Olivier RISSER-MAROIX

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce fichier ipython complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un second envoi peut être fait **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyez le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

#### A télécharger

Avant de commencer, télécharger l'archive **tme-09.tgz** à l'adresse indiquée.

Une fois détarée, cette archive crée un répertoire contenant :
- deux fichiers exécutables (Linux): *apriori32* et *fpgrowth32* (ces 2 programmes ont été développés par Christian Borgelt, chercheur au ``European Center of Soft Computing" , plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance.
- un répertoire *datasets* contenant des bases d'exemples pour tester les algorithmes
- ce fichier ipython notebook que vous lisez en ce moment et que vous allez compléter.



## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier "*exemple-1.txt*" (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [4]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font size="+1">**[Q]**</font> Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble d'items plutôt que comme une liste. Il sera ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [5]:
def chargeBase(path, delimiter=' '):
    f = open(path, 'r')
    lecteur = csv.reader(f, delimiter=delimiter)
    out = {i: set(line) for i, line in enumerate(lecteur)}
    f.close()
    return out

In [6]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font size="+1">**[Q]**</font> Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [7]:
def noms_items(base):
    out = set()
    for k in base:
        out = out.union(base[k])
    return out

In [8]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font size="+1">**[Q]**</font> Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

Remarque: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [9]:
def singletons(base):
    return [{c} for c in noms_items(base)]

In [10]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'c'}, {'d'}, {'b'}, {'a'}, {'e'}]

<font size="+1">**[Q]**</font> Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [11]:
def comptage(base, itemset):
    c = 0
    for k in base:
        if base[k].intersection(itemset) == itemset:
            c += 1
    return c

In [12]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font size="+1">**[Q]**</font> Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [13]:
def support(base, itemset):
    return comptage(base, itemset) / len(base)

In [14]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : "+str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [15]:
def decompose(s):
    s = list(sorted(s))
    return [set(s[:i] + s[i+1:]) for i in range(len(s))]
    """L = []
    for i in range(len(s)):
        L.append(set(s[:i] + s[i+1:]))
    return L"""

#print(decompose(set({'a', 'b', 'c'})))

def apriori_gen(prev):
    ln = len(prev[0]) + 1
    L = []
    for it1 in prev:
        for it2 in prev:
            it3 = it1.union(it2)
            if len(it3) == ln:
                f = True
                for d in decompose(it3):
                    if not d in prev:
                        f = False
                        break
                if f:
                    L.append(tuple(sorted(it3)))
    L = set(L)
    L = list(map(lambda t: set(t), L))
    return L

In [16]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'d', 'c'}, {'b', 'c'}, {'a', 'd'}, {'b', 'd'}, {'a', 'c'}]

Exemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend 
[{'b', 'd', 'c'}, {'b', 'a', 'd'}]


<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [26]:
def apriori(base, min_support):
    out = []
    candidats = singletons(base)
    supports  = list(map(lambda it: support(base, it), candidats))
    c_s = list(filter(lambda c: c[1] >= min_support, zip(candidats, supports)))
    candidats, supports = zip(*c_s) if len(c_s) else ([], [])
    for c, s in c_s:
        out.append((c, s))
    print(out)
    while candidats:
        candidats = apriori_gen(candidats)
        supports  = list(map(lambda it: support(base, it), candidats))
        c_s = list(filter(lambda c: c[1] >= min_support, zip(candidats, supports)))
        candidats, supports = zip(*c_s) if len(c_s) else ([], [])
        for c, s in c_s:
            out.append((c, s))
    return out

In [27]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'c'}, 0.7), ({'d'}, 0.7), ({'b'}, 0.7), ({'a'}, 0.6), ({'e'}, 0.3)]
[({'c'}, 0.7), ({'d'}, 0.7), ({'b'}, 0.7), ({'a'}, 0.6), ({'e'}, 0.3), ({'b', 'a'}, 0.5), ({'d', 'c'}, 0.4), ({'b', 'c'}, 0.6), ({'d', 'e'}, 0.3), ({'a', 'd'}, 0.4), ({'b', 'd'}, 0.4), ({'a', 'c'}, 0.4), ({'b', 'd', 'c'}, 0.3), ({'b', 'a', 'd'}, 0.3), ({'b', 'a', 'c'}, 0.4)]


In [19]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'c'}, 0.7), ({'d'}, 0.7), ({'b'}, 0.7), ({'a'}, 0.6), ({'e'}, 0.3)]
({'c'}, {'d'}, {'b'}, {'a'}, {'e'})
(0.7, 0.7, 0.7, 0.6, 0.3)
[({'c'}, 0.7), ({'d'}, 0.7), ({'b'}, 0.7), ({'a'}, 0.6), ({'e'}, 0.3), ({'b', 'a'}, 0.5), ({'d', 'c'}, 0.4), ({'b', 'c'}, 0.6), ({'d', 'e'}, 0.3), ({'a', 'd'}, 0.4), ({'b', 'd'}, 0.4), ({'a', 'c'}, 0.4), ({'b', 'd', 'c'}, 0.3), ({'b', 'a', 'd'}, 0.3), ({'b', 'a', 'c'}, 0.4)]


## Expérimentations

### Vérification avec la base du TD

Utilisez votre fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que vous obtenez les bons itemsets fréquents.

In [20]:
a, b, c, d, e = 'a', 'b', 'c', 'd', 'e'
base_td = {
    0: {a, b, d, e},
    1: {b, c, d},
    2: {a, b, d, e},
    3: {a, c, d, e},
    4: {b, c, d, e},
    5: {b, d, e},
    6: {c, d},
    7: {a, b, c},
    8: {a, d, e},
    9: {b, d}
}
apriori(base_td, 0.3)

[({'c'}, 0.5), ({'d'}, 0.9), ({'b'}, 0.7), ({'a'}, 0.5), ({'e'}, 0.6)]
({'c'}, {'d'}, {'b'}, {'a'}, {'e'})
(0.5, 0.9, 0.7, 0.5, 0.6)


[({'c'}, 0.5),
 ({'d'}, 0.9),
 ({'b'}, 0.7),
 ({'a'}, 0.5),
 ({'e'}, 0.6),
 ({'a', 'b'}, 0.3),
 ({'b', 'e'}, 0.4),
 ({'c', 'd'}, 0.4),
 ({'b', 'c'}, 0.3),
 ({'a', 'e'}, 0.4),
 ({'d', 'e'}, 0.6),
 ({'a', 'd'}, 0.4),
 ({'b', 'd'}, 0.6),
 ({'b', 'd', 'e'}, 0.4),
 ({'a', 'd', 'e'}, 0.4)]

In [21]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
apriori(base_td, 0.3)

Exemple: apriori(BaseTD, 0.3) rend 
[({'c'}, 0.5), ({'d'}, 0.9), ({'b'}, 0.7), ({'a'}, 0.5), ({'e'}, 0.6)]
({'c'}, {'d'}, {'b'}, {'a'}, {'e'})
(0.5, 0.9, 0.7, 0.5, 0.6)


[({'c'}, 0.5),
 ({'d'}, 0.9),
 ({'b'}, 0.7),
 ({'a'}, 0.5),
 ({'e'}, 0.6),
 ({'a', 'b'}, 0.3),
 ({'b', 'e'}, 0.4),
 ({'c', 'd'}, 0.4),
 ({'b', 'c'}, 0.3),
 ({'a', 'e'}, 0.4),
 ({'d', 'e'}, 0.6),
 ({'a', 'd'}, 0.4),
 ({'b', 'd'}, 0.6),
 ({'b', 'd', 'e'}, 0.4),
 ({'a', 'd', 'e'}, 0.4)]

### Comparaisons avec les implémentations apriori32 et fpgrowth

<font size="+1">**[Q]**</font> Tester votre programme sur les 3 bases données dans datasets et confronter vos résultats avec le programme *apriori32*.

In [22]:
apriori(chargeBase('datasets/titanic-red.csv', ','), .1)

[({'dead'}, 0.5344129554655871), ({'3rd'}, 0.27125506072874495), ({'male'}, 0.6099865047233468), ({'Cherbourg'}, 0.23751686909581646), ({'alive'}, 0.4642375168690958), ({'Southampton'}, 0.7112010796221323), ({'2nd'}, 0.3481781376518219), ({'female'}, 0.38866396761133604), ({'1st'}, 0.3792172739541161)]
({'dead'}, {'3rd'}, {'male'}, {'Cherbourg'}, {'alive'}, {'Southampton'}, {'2nd'}, {'female'}, {'1st'})
(0.5344129554655871, 0.27125506072874495, 0.6099865047233468, 0.23751686909581646, 0.4642375168690958, 0.7112010796221323, 0.3481781376518219, 0.38866396761133604, 0.3792172739541161)


[({'dead'}, 0.5344129554655871),
 ({'3rd'}, 0.27125506072874495),
 ({'male'}, 0.6099865047233468),
 ({'Cherbourg'}, 0.23751686909581646),
 ({'alive'}, 0.4642375168690958),
 ({'Southampton'}, 0.7112010796221323),
 ({'2nd'}, 0.3481781376518219),
 ({'female'}, 0.38866396761133604),
 ({'1st'}, 0.3792172739541161),
 ({'Southampton', 'male'}, 0.46288798920377866),
 ({'Cherbourg', 'female'}, 0.11336032388663968),
 ({'1st', 'Southampton'}, 0.20917678812415655),
 ({'1st', 'female'}, 0.1592442645074224),
 ({'Southampton', 'dead'}, 0.41295546558704455),
 ({'2nd', 'alive'}, 0.15789473684210525),
 ({'2nd', 'female'}, 0.1417004048582996),
 ({'2nd', 'dead'}, 0.1902834008097166),
 ({'2nd', 'Southampton'}, 0.30634278002699056),
 ({'2nd', 'male'}, 0.20647773279352227),
 ({'Cherbourg', 'alive'}, 0.14709851551956815),
 ({'3rd', 'Southampton'}, 0.19568151147098514),
 ({'1st', 'male'}, 0.21997300944669365),
 ({'Southampton', 'alive'}, 0.2982456140350877),
 ({'1st', 'alive'}, 0.23076923076923078),
 ({'1st', 

<font size="+1">**[Q]**</font> Utiliser le programme *apriori32* pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [23]:
apriori(chargeBase('datasets/mushrooms.txt', ','), .3)

[({'NONE'}, 0.45817490494296575), ({'BULBOUS'}, 0.4515209125475285), ({'NARROW'}, 0.30133079847908745), ({'BROWN'}, 0.5881653992395437), ({'FIBROUS'}, 0.37250475285171103), ({'SCALY'}, 0.3911596958174905), ({'FLAT'}, 0.3911596958174905), ({'FREE'}, 0.9743346007604563), ({'WOODS'}, 0.3754752851711027), ({'BROAD'}, 0.6986692015209125), ({'PENDANT'}, 0.4714828897338403), ({'SILKY'}, 0.3431558935361217), ({'PARTIAL'}, 1.0), ({'ENLARGING'}, 0.4220532319391635), ({'SMOOTH'}, 0.7873098859315589), ({'BRUISES'}, 0.4011406844106464), ({'CLOSE'}, 0.8108365019011406), ({'TAPERING'}, 0.5779467680608364), ({'NO'}, 0.5988593155893536), ({'EVANESCENT'}, 0.36311787072243346), ({'WHITE'}, 0.9771863117870723), ({'SEVERAL'}, 0.4828897338403042), ({'EDIBLE'}, 0.5332699619771863), ({'CONVEX'}, 0.45104562737642584), ({'PINK'}, 0.49667300380228135), ({'ONE'}, 0.9230038022813688), ({'POISONOUS'}, 0.4667300380228137), ({'BUFF'}, 0.31653992395437264), ({'GRAY'}, 0.37975285171102663)]
({'NONE'}, {'BULBOUS'}, {'NA

[({'NONE'}, 0.45817490494296575),
 ({'BULBOUS'}, 0.4515209125475285),
 ({'NARROW'}, 0.30133079847908745),
 ({'BROWN'}, 0.5881653992395437),
 ({'FIBROUS'}, 0.37250475285171103),
 ({'SCALY'}, 0.3911596958174905),
 ({'FLAT'}, 0.3911596958174905),
 ({'FREE'}, 0.9743346007604563),
 ({'WOODS'}, 0.3754752851711027),
 ({'BROAD'}, 0.6986692015209125),
 ({'PENDANT'}, 0.4714828897338403),
 ({'SILKY'}, 0.3431558935361217),
 ({'PARTIAL'}, 1.0),
 ({'ENLARGING'}, 0.4220532319391635),
 ({'SMOOTH'}, 0.7873098859315589),
 ({'BRUISES'}, 0.4011406844106464),
 ({'CLOSE'}, 0.8108365019011406),
 ({'TAPERING'}, 0.5779467680608364),
 ({'NO'}, 0.5988593155893536),
 ({'EVANESCENT'}, 0.36311787072243346),
 ({'WHITE'}, 0.9771863117870723),
 ({'SEVERAL'}, 0.4828897338403042),
 ({'EDIBLE'}, 0.5332699619771863),
 ({'CONVEX'}, 0.45104562737642584),
 ({'PINK'}, 0.49667300380228135),
 ({'ONE'}, 0.9230038022813688),
 ({'POISONOUS'}, 0.4667300380228137),
 ({'BUFF'}, 0.31653992395437264),
 ({'GRAY'}, 0.37975285171102663),


<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font size="+1">**[Q]**</font>  Utiliser *fpgrowth32* pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

<font size="+1">**[Q]**</font> En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des programmes *apriori32* et *fpgrowth32*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori32*, soit *fpgrowth32*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth32 -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
